In [1]:
import pickle, json, time, datetime, pymongo, requests, copy, os, csv
from pymongo import MongoClient
import pandas as pd
from Bio import Medline
from Bio import Entrez

In [3]:
now=datetime.datetime.now()

In [5]:
now_time=now.strftime('%Y_%m_%d')

In [6]:
print(now_time)

2023_05_09


In [10]:
def cal_time(time):
    time = time.split('/')
    year = int(time[0])
    month = int(time[1]) - 3
    day = int(time[2])
    if month < 1:
        month = 12
        year -= 1
    time = datetime.datetime(year=year, month=month, day=1).strftime('%Y/%m/%d')
    return time

In [13]:
former_time=cal_time(nowtime)

In [14]:
former_time

'2023/02/01'

In [16]:
terms='("aging"[Title/Abstract] OR "ageing"[Title/Abstract] OR "longevity"[Title/Abstract] OR "centenarian"[Title/Abstract] OR "centenarians"[Title/Abstract] \
OR "the elderly"[Title/Abstract] OR "the aged"[Title/Abstract] OR "old people"[Title/Abstract] OR "older people"[Title/Abstract] OR "old age"[Title/Abstract]  AND \
Journal Article[ptyp] AND "humans"[MeSH Terms] AND English[lang] AND ("'+str(former_time)+'"[Date - Publication] : "3000"[Date - Publication]))'
Entrez.email="12007002@zju.edu.cn"
handle_0 = Entrez.esearch(db="pubmed", term=terms,retmax=1000000)
record = Entrez.read(handle_0)
idlist = record["IdList"]
No_Papers = len(idlist)
print(No_Papers)

4733


In [19]:
len(idlist)

4733

In [20]:
len(set(idlist))

4733

In [21]:
idlist[0]

'37149695'

In [22]:
res=[]
for i in range(0,len(idlist),10000):
    id=idlist[i:i+10000]
    handle_1 = Entrez.efetch(db="pubmed", id=id, rettype="medline",retmode="text")
    records = Medline.parse(handle_1)
    res.append(list(records))

In [23]:
raw_pick=[]
for i in res:
    raw_pick+=i

In [24]:
len(raw_pick)

4733

In [30]:
client = MongoClient(username="hald",password="mclab236")

In [31]:
db = client.hald

# pubmed_raw_pmid

In [32]:
collections_pubmed_raw_pmid=db.pubmed_raw_pmid

In [33]:
pmids=list(collections_pubmed_raw_pmid.find())

In [34]:
pmids[:2]

[{'_id': ObjectId('6393476d5303aaecac1f079a'),
  'PMID': '35796512',
  'addtime': '20220724'},
 {'_id': ObjectId('6393476d5303aaecac1f079b'),
  'PMID': '35796396',
  'addtime': '20220724'}]

In [35]:
pmid_list=[]
for i in pmids:
    pmid_list.append(i.get('PMID'))

In [36]:
pmid_list[:2]

['35796512', '35796396']

In [37]:
len(pmid_list)

292974

In [38]:
pmid_dict={}
for i in pmids:
    pmid_dict[i.get('PMID')]=i.get('PMID')

In [39]:
len(pmid_dict)

292974

In [40]:
len(raw_pick)

4733

In [42]:
reserved_raw_pick=[]
reserved_raw_pick_pmid=[]
for i in raw_pick:
    if i.get('AB'):
        if not pmid_dict.get(i.get('PMID')):
            i['addtime']=now_time
            reserved_raw_pick.append(i)
            reserved_raw_pick_pmid.append({'PMID':i.get('PMID'),'addtime':i.get('addtime')})

In [43]:
len(reserved_raw_pick)

2605

In [44]:
collections_pubmed_raw_pmid.insert_many(reserved_raw_pick_pmid)

In [45]:
len(list(collections_pubmed_raw_pmid.find()))

295579

# pubmed_raw (reserved_raw_pick)

In [47]:
collections_pubmed_raw = db.pubmed_raw

In [48]:
collections_pubmed_raw.insert_many(reserved_raw_pick)

In [109]:
reserved_raw_pick[20]

{'PMID': '37143978',
 'OWN': 'NLM',
 'STAT': 'MEDLINE',
 'DCOM': '20230508',
 'LR': '20230508',
 'IS': '2296-2565 (Electronic) 2296-2565 (Linking)',
 'VI': '11',
 'DP': '2023',
 'TI': 'Impact of community care services on the health of older adults: evidence from China.',
 'PG': '1160151',
 'LID': '10.3389/fpubh.2023.1160151 [doi] 1160151',
 'AB': 'INTRODUCTION: The rapid growth in the population of older adults has put tremendous pressure on medical and social services in countries including China. Community care services are a feasible solution for promoting healthy aging in developing countries. This study investigated the association between community care services and the health of older adults in China. METHOD: Using nationally representative survey data from China, consisting of four waves conducted in 2005, 2008, 2011, and 2014, a balanced panel dataset was constructed using a sample of 4,700 older adults (33.1% aged 80 years or older, 51.0% residing in rural areas, and 48.8% w

# pubtator_raw_pmid

In [49]:
collection_pubtator_raw_pmid = db.pubtator_raw_pmid

In [50]:
previous_pubtator_pmid=list(collection_pubtator_raw_pmid.find({}))

In [51]:
len(previous_pubtator_pmid)

270644

In [52]:
previous_pubtator_pmids_dict={}
for i in previous_pubtator_pmid:
    previous_pubtator_pmids_dict[i.get('PMID')]=i.get('PMID')

In [53]:
current_pubtator_pmid=[]
for i in raw_pick:
    if i.get('AB'):
        if not previous_pubtator_pmids_dict.get(i.get('PMID')):
            i['addtime']=now_time
            current_pubtator_pmid.append(i.get('PMID'))

In [54]:
len(current_pubtator_pmid)

2654

In [55]:
pmids_list=[]
for i in current_pubtator_pmid:
    pmids_list.append(i)

In [56]:
len(pmids_list)

2654

In [58]:
def SubmitPMIDList(pmids,Format,now_time):
	json = {"pmids": pmids}
	r = requests.post("https://www.ncbi.nlm.nih.gov/research/pubtator-api/publications/export/"+Format , json = json)
	if r.status_code != 200 :
		print ("[Error]: HTTP code "+ str(r.status_code))
	else:
		with open(now_time+'_pubtator_raw.txt','a',encoding="UTF-8") as fileread:
			fileread.write(str(r.text.encode("utf-8"),encoding="UTF-8"))

In [59]:
import time
pubtator_raw=[]
for i in range(0,len(pmids_list),1000):
    SubmitPMIDList(pmids_list[i:i+1000],'biocjson',now_time)
    time.sleep(0.1)

In [60]:
ttt=open(now_time+'_pubtator_raw.txt',"r")

In [61]:
overjson=[]
for l in ttt.readlines():
    overjson.append(l.rstrip())

In [63]:
len(overjson)

2100

In [65]:
pubtator_list=copy.deepcopy(overjson)

In [66]:
pubtator_json=[]
for i in pubtator_list:
    pubtator_json.append(json.loads(i))

In [67]:
pubtator_pmid_list=[]
for i in pubtator_json:
    pubtator_pmid_list.append(i.get('id'))

In [68]:
len(pubtator_pmid_list)

2100

In [73]:
pubtator_pmid_list[0]

'37060077'

In [74]:
current_pubtator_pmid_list=[]
for i in pubtator_pmid_list:
    current_pubtator_pmid_list.append({'PMID':i,'addtime':now_time})

In [75]:
len(current_pubtator_pmid_list)

2100

In [76]:
collection_pubtator_raw_pmid.insert_many(current_pubtator_pmid_list)

# pubtator_info  (raw_pick_pubtator)

In [77]:
collections_pubtator_info = db.pubtator_info

In [78]:
filtered_pubtator_json=[]
for i in pubtator_json:
    if i.get('passages'):
        for j in i.get('passages')[1].get('annotations'):
            if j.get('infons'):
                if j.get('infons').get('identifier')=='9606':
                    filtered_pubtator_json.append(i)
                    break

In [79]:
len(filtered_pubtator_json)

1597

In [80]:
second_filtered_pubtator_json=[]
for i in filtered_pubtator_json:
    if i.get('passages'):
        for j in i.get('passages')[1].get('annotations'):
            if j.get('infons'):
                    if j.get('infons').get('type') in ('Gene', 'Chemical', 'Mutation','Disease') :
                        second_filtered_pubtator_json.append(i)
                        break

In [81]:
len(second_filtered_pubtator_json)

1402

In [82]:
entities_dict={}
for i in second_filtered_pubtator_json:
    if i.get('id'):
        entities_list=[]
        for j in i.get('passages')[1].get('annotations'):
            entities_list.append(j.get('text'))
        entities_dict[i.get('id')]=entities_list

In [83]:
second_filtered_pubtator_dict={}
for i in second_filtered_pubtator_json:
    if i.get('id'):
        second_filtered_pubtator_dict[i.get('id')]=i

In [85]:
second_filtered_pubtator_json[0]

{'_id': '37060077|None',
 'id': '37060077',
 'infons': {},
 'passages': [{'infons': {'journal': 'Environ Health;2023Apr14; 22 (1) 35. doi:10.1186/s12940-023-00986-9',
    'year': '2023',
    'type': 'title',
    'authors': 'Shim JI, Byun G, Lee JT, ',
    'section': 'Title'},
   'offset': 0,
   'text': "Long-term exposure to particulate matter and risk of Alzheimer's disease and vascular dementia in Korea: a national population-based Cohort Study.",
   'sentences': [],
   'annotations': [{'id': '2',
     'infons': {'identifier': 'MESH:D000544', 'type': 'Disease'},
     'text': "Alzheimer's disease",
     'locations': [{'offset': 53, 'length': 19}]},
    {'id': '3',
     'infons': {'identifier': 'MESH:D015140', 'type': 'Disease'},
     'text': 'vascular dementia',
     'locations': [{'offset': 77, 'length': 17}]}],
   'relations': []},
  {'infons': {'type': 'abstract', 'section': 'Abstract'},
   'offset': 147,
   'text': "BACKGROUND: The prevalence of age-related neurodegenerative disea

In [86]:
from nltk.tokenize import sent_tokenize
import datetime
import re
from multiprocessing import Process, Manager, Pool
import os, time, random
def work(i,rawlength,lock):
    info=i.get('passages')[1]
    abstract=info.get('text')
    sentences=sent_tokenize(abstract)
    positions=[]
    size=len(sentences)
    start=0
    for k,j in enumerate(sentences):
        newj=j.translate(str.maketrans({"-":  r"\-","\\": r"\\", "^":  r"\^","$":  r"\$","*":  r"\*",".":  r"\.","(":  r"\(",")":  r"\)","+":  r"\+","[":  r"\[","]":  r"\]",
                                      "{":  r"\{","}":  r"\}","|":  r"\|","?":  r"\?"}))
        spanlist=re.search(newj,abstract,flags=re.M|re.I).span()
        abstract=abstract[spanlist[1]:]
        positions.append((start+spanlist[0],start+spanlist[1]))
        start+=spanlist[1]
    middict={'PMID':i.get('id'),'AB':info.get('text'),'SENTENCE':sentences,'positions':positions,'annotations':info.get('annotations'),'offset':info.get('offset')}
    lock.acquire()
    rawlength.append(middict)
    lock.release()
if __name__=='__main__':
    start=datetime.datetime.now()
    print('Parent process %s.' % os.getpid())
    p=Pool(40)
    manager=Manager()
    rawlength=manager.list()
    lock=manager.Lock()
    for i in second_filtered_pubtator_json:
        if i.get('passages'):
            p.apply_async(work, args=(i,rawlength,lock))
    print('Waiting for all subprocesses done...')
    p.close()
    p.join()
    print('All subprocesses done.')
    end=datetime.datetime.now()
    print("The running time is "+str((end-start).seconds)+"s")  

Parent process 66008.
Waiting for all subprocesses done...
All subprocesses done.
The running time is 7s


In [87]:
pubtators_list=list(rawlength)

In [88]:
len(pubtators_list)

1402

In [89]:
pubtator_raw=[]
error=[]
error2=[]
for i in pubtators_list:
    for j in i.get('annotations'):
        for k,l in enumerate(i.get('positions')):
            if j.get('locations')[0].get('offset')>=l[0]+i.get('offset') and j.get('locations')[0].get('offset')<l[1]+i.get('offset'):
                pubtator_raw.append({'id':i.get('PMID'),'AB':i.get('AB'), 'type': j.get('infons').get('type'), \
                                     'identifier':j.get('infons').get('identifier'),'text':j.get('text'),'sentence':i.get('SENTENCE')[k],\
                                     'sentenceSize':len(i.get('SENTENCE')),'sentenceIndex':k})
                break
        else:
            error.append(i)
            error2.append(j)

In [90]:
pubtator_raw_dict2AB={}
for i in pubtator_raw:
    pubtator_raw_dict2AB[i.get('id')]=i.get('AB')

In [91]:
len(error)

0

In [92]:
len(error2)

0

In [93]:
len(pubtator_raw)

20195

In [94]:
raw_pick_pubtator=[]
for i in reserved_raw_pick:
    if i.get('AB'):
        if pubtator_raw_dict2AB.get(i.get('PMID')):
            i.update({'AB':pubtator_raw_dict2AB.get(i.get('PMID'))})
            raw_pick_pubtator.append(i)

In [95]:
len(pubtator_raw_dict2AB)

1402

In [96]:
len(reserved_raw_pick)

2605

In [97]:
len(raw_pick_pubtator)

1402

In [98]:
for i in raw_pick_pubtator:
    i['addtime']=now_time

In [99]:
collections_pubtator_info.insert_many(raw_pick_pubtator)

In [100]:
len(list(collections_pubtator_info.find()))

175189

# pubtator_for_compare_latest (pubtator_raw)

In [101]:
collections_pubtator_for_compare_latest=db.pubtator_for_compare_latest

In [102]:
for i in pubtator_raw:
    i['addtime']=now_time

In [103]:
collections_pubtator_for_compare_latest.insert_many(pubtator_raw)

In [104]:
len(pubtator_raw)

20195

In [105]:
pwd

'/root/hald_autoupdate/2023_05_09'

In [106]:
pickle.dump(raw_pick,open('results/step1/raw_pick.pkl','wb'))
pickle.dump(raw_pick_pubtator,open('results/step1/raw_pick_pubtator.pkl','wb'))
pickle.dump(pubtator_raw,open('results/step1/pubtator_raw.pkl','wb'))

In [107]:
pubtator_raw[0]

{'id': '37058495',
 'AB': "Despite the rising global burden of stroke and its socio-economic implications, the neuroimaging predictors of subsequent cognitive impairment are still poorly understood. We address this issue by studying the relationship of white matter integrity assessed within ten days after stroke and patients' cognitive status one year after the attack. Using diffusion-weighted imaging, we apply the Tract-Based Spatial Statistics analysis and construct individual structural connectivity matrices by employing deterministic tractography. We further quantify the graph-theoretical properties of individual networks. The Tract-Based Spatial Statistic did identify lower fractional anisotropy as a predictor of cognitive status, although this effect was mostly attributable to the age-related white matter integrity decline. We further observed the effect of age propagating into other levels of analysis. Specifically, in the structural connectivity approach we identified pairs of 

In [108]:
now_time

'2023_05_09'

In [ ]:
os.makedirs('results/step1/')
os.makedirs('results/step2/')
os.makedirs('results/step3/')
os.makedirs('results/step4/')
os.makedirs('results/step5/')

# Literature_Info.json

In [119]:
pd.set_option('display.max_columns', 50)

In [120]:
jcr=pd.read_csv("../JCR/jcr2022.tsv",sep="\t")

In [121]:
len(jcr)

21430

In [122]:
jcr[-10:-1]

,rank,journalTitle,abbrJournal,year,edition,issn,totalCites,journalImpactFactor,impactFactorWithoutJournalSelfCites,fiveYearImpactFactor,immediacyIndex,citableItems,citedHalfLife,citingHalfLife,eigenFactorScore,articleInfluenceScore,citesCurrent,articlesCurrent,selfCites,impactFactorSelfCites,originalResearch,jifPercentile,normEigenFactor
21420,12425,AGENDA,AGENDA,2022,NaN,0002-0796,159,0.0,0.0,0.0,0.0,0,17.8,0.0,0.00005,0.114,NaN,NaN,NaN,NaN,NaN,0.0,0.01206
21421,12425,Abakos,ABAKOS,2022,NaN,2316-9451,7,0.0,0.0,0.0,0.0,18,0.0,10.7,0.00003,0.068,NaN,NaN,NaN,NaN,NaN,0.0,0.00733
21422,12425,Adalya,ADALYA,2022,NaN,1301-2746,75,0.0,0.0,0.0,0.0,19,0.0,16.2,0.00027,0.306,NaN,NaN,NaN,NaN,NaN,0.0,0.05846
21423,12425,Anduli,ANDULI,2022,NaN,1696-0270,35,0.0,0.0,0.0,0.0,15,0.0,14.6,0.00007,0.150,NaN,NaN,NaN,NaN,NaN,0.0,0.01555
21424,12425,Anemia,ANEMIA,2022,NaN,2090-1267,502,0.0,0.0,0.0,0.0,14,7.3,7.7,0.00030,0.550,NaN,NaN,NaN,NaN,NaN,0.0,0.06589
21425,12425,Appeal,APPEAL,2022,NaN,1205-612X,56,0.0,0.0,0.0,0.0,6,0.0,4.8,0.00019,0.550,NaN,NaN,NaN,NaN,NaN,0.0,0.04134
21426,12425,Atenas,ATENAS,2022,NaN,1682-2749,60,0.0,0.0,0.0,0.0,0,0.0,0.0,0.00007,0.034,NaN,NaN,NaN,NaN,NaN,0.0,0.01561
21427,12425,Atenea,ATENEA-CONCEPC,2022,NaN,0718-0462,105,0.0,0.0,0.0,0.0,35,8.8,15.8,0.00034,0.281,NaN,NaN,NaN,NaN,NaN,0.0,0.07463
21428,12425,AusArt,AUSART,2022,NaN,2340-9134,13,0.0,0.0,0.0,0.0,32,0.0,11.8,0.00002,0.017,NaN,NaN,NaN,NaN,NaN,0.0,0.00594


In [123]:
jt2if_all={}
for i in range(len(jcr)):
    if jt2if_all.get(jcr.iloc[i,1].lower()):
        if jt2if_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
jt2if_all['aging']=jt2if_all.get('aging-us')

In [124]:
jt2if_brief={}
for i in range(len(jcr)):
    if jt2if_brief.get(jcr.iloc[i,2].lower()):
        if jt2if_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
jt2if_brief['aging']=jt2if_brief.get('aging-us')

In [125]:
list(jt2if_brief.items())[:2]

[('ca-cancer j clin', 286.13), ('lancet', 202.731)]

In [126]:
len(jt2if_brief)

21429

In [127]:
jt2if5_all={}
for i in range(len(jcr)):
    if jt2if5_all.get(jcr.iloc[i,1].lower()):
        if jt2if5_all.get(jcr.iloc[i,1].lower())==0.0:
            jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_all[jcr.iloc[i,1].lower()]=jcr.iloc[i,9]
jt2if5_all['aging']=jt2if5_all.get('aging-us')

In [128]:
jt2if5_brief={}
for i in range(len(jcr)):
    if jt2if5_brief.get(jcr.iloc[i,2].lower()):
        if jt2if5_brief.get(jcr.iloc[i,2].lower())==0.0:
            jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,7]
    else:
        jt2if5_brief[jcr.iloc[i,2].lower()]=jcr.iloc[i,9]
jt2if5_brief['aging']=jt2if5_brief.get('aging-us')

In [114]:
Literature_Info_list=list(collections_pubmed_raw.find())

In [115]:
len(Literature_Info_list)

295579

In [129]:
Literature_Info_dict={}
for i in Literature_Info_list:
    if jt2if_brief.get(i.get('TA').lower()):
        i['IF']=jt2if_brief.get(i.get('TA').lower())
    elif jt2if_all.get(i.get('JT').lower()):
        i['IF']=jt2if_all.get(i.get('JT').lower())
    else:
        i['IF']=0.0
    if jt2if5_brief.get(i.get('TA').lower()):
        i['IF5']=jt2if5_brief.get(i.get('TA').lower())
    elif jt2if5_all.get(i.get('JT').lower()):
        i['IF5']=jt2if5_all.get(i.get('JT').lower())
    else:
        i['IF5']=0.0
    Literature_Info_dict[i.get('PMID')]={
    'PMID':i.get('PMID'),
        'TI':i.get('TI'),
        'AB':i.get('AB'),
        'IF':i.get('IF'),
        'IF5':i.get('IF5'),
        'AU':i.get('AU'),
        'FAU':i.get('FAU'),
        'AD':i.get('AD'),
        'PT':i.get('PT'),
        'DP':i.get('DP'),
        'PL':i.get('PL'),
        'JT':i.get('JT'),
        'TA':i.get('TA'),
        'SO':i.get('SO')
    }

In [130]:
list(Literature_Info_dict.items())[10]

('35795148',
 {'PMID': '35795148',
  'TI': 'Sirtuin 1 in Chronic Kidney Disease and Therapeutic Potential of Targeting Sirtuin 1.',
  'AB': 'The incidence and prevalence of chronic kidney disease (CKD) continue to increase worldwide remaining as a major public health burden. CKD eventually progresses to end-stage kidney failure and patients with CKD have high morbidity and mortality. Sirtuin 1 (SIRT1), a NAD+-dependent deacetylases, has significant renal protective effects through its regulation of fibrosis, apoptosis, and senescence, oxidative stress, inflammation and aging process. The renal protective effects of Sirt1 have been described in many kidney diseases such as diabetic kidney disease and HIV-related kidney disease. SIRT1 also has protective effects against vascular calcification and therefore could be developed as a therapy for both CKD and CKD complications. In this narrative review, we will give an overview of the recent progress on the role of SIRT1 and its downstream pa

In [131]:
json.dump(Literature_Info_dict,open('results/step5/Literature_Info.json','w'))